In [1]:
import math
import numpy as np
from sklearn import datasets
import pandas as pd

## We'll be implementing given standard functions for our Decision Tree Classifier:
### 1.Entropy Function 
### 2.Information Gain Function
### 3.Split Info function
### 4.Best feature and Maximum Gain Function
### 5.Decision Tree Function



In [2]:
def entropy(y):
    #entropy is -(summation of p(ith)logp(ith)) for i classes
    #note: We have assumed target/y has 1 column only.
    
    total_elements = len(y)  
    entropy_value = 0  
    
    labels_in_y= np.unique(y)
    
    for l in labels_in_y:  
        number_of_ith_class = len(y[y== l])   
        prob_of_ith_class = number_of_ith_class/total_elements     
        entropy_value += prob_of_ith_class * math.log(prob_of_ith_class, 2)  
        
    entropy_value = -entropy_value
    
    return entropy_value

In [3]:
def information_gain(x, y, feature) :
    
    #information gain is= entropy before splitting - entropy after splitting
    #entropy after splitting is summation of weighted entropies
    
    entropy_before_split= entropy(y)  
    total_elements = len(y)   
    entropy_after_split = 0   
    
    #x[feature] gives data points /column for the selected feature
    #unique_values gives the total no of labels available in this feature 
    unique_values = set(x[feature])  
    
    #our splitting is that for given feature, no of splits is = no of unique values in this feature
    for i in unique_values:
        child = y[(x[feature] == i)]
        i_entropy = entropy(child)
        
        #finding summation of weighted entropies
        entropy_after_split += (i_entropy * len(child)) / total_elements
        
    info_gain = entropy_before_split - entropy_after_split  
    
    return info_gain

In [4]:
def split_information(x, y, feature) :
    #split info is negative summation of weighted logarithms of weighted avgs
    total_elements= len(y)
    SI= 0
    unique_values = set(x[feature])
    
    for i in unique_values :  
        
        child = y[x[feature] == i] 
        
        weighted_avg_for_this_child= len(child) / total_elements
        w=weighted_avg_for_this_child
        
        SI +=  w* math.log(w, 2) 
        
    return -1 * SI


In [5]:
def best_split_possible(x,y,features):
    #we need to find best feature and max gain in order to find best split possible
    #max gain means max information gain found
    #initializing best feature and max gain
    best_feature = '' 
    max_gain_ratio = -1  
    
    #we need to select feature with max gain ratio
    #gain ratio is =info gain/split info
    
    for f in features :
        
        
        split_info = split_information(x, y, f)
        info_gain = information_gain(x, y, f)
        
        if split_info != 0 :
            gain_ratio = info_gain / split_info
            
        else :
            #this would be the case when split info is 0
            gain_ratio = -1
        
        
        ####
        if gain_ratio > max_gain_ratio :
            best_feature = f
            max_gain_ratio = gain_ratio
        ####    
    
    return best_feature, max_gain_ratio
    
    

## Decision Tree

In [6]:
def DT(x,y,features,level):
    
    #classes will be the classes available to use for current y
    classes=np.unique(y)   
    #now we will code our base cases
    
    ######
    # base case 1, we have leaf node 
    if (len(classes) == 1): 
            
        print("Level ",level)   
        c = list(classes)[0]    
        
        print("Count of ",c," = ",len(y))   
        #leaf node has zero entropy
        print("Current Entropy is = 0.0")                 
        print("Reached leaf Node")                          
        print("Output Class Name =", c)
        print()
        return
    ######
    
    
    
    ######
    # base case 2
    #if we have 0 features, no further splitting possible
    elif len(features)==0:                                                            
        print("Level  ",level)     
        countOfClasses=[]
        #printing count of all classes available to us currently
        for currently_available_class in classes:
            count_of_currently_avail_class = (y == currently_available_class).sum() 
            countOfClasses.append(count_of_currently_avail_class)
            print("Count of ",currently_available_class," = ",count_of_currently_avail_class) 
            
        entropy_current = entropy(y)
        
        #we will print the majority class
        #finding index of majority class
        i=countOfClasses.index(max(countOfClasses))
        
        print("Current Entropy is = ",entropy_current)
        print("No more features left")
        print("Reached leaf Node")
        print("Output Class Name =", classes[i])
        print()
        return
    ######
    
    
    
    #################################################
    # Working of DT 
    else:
        
        print("Level ",level)        
        
        #we'll find the class with maximum data pts and take it as our output class
        max_count = 0
        for current_class in classes:             
            countOfCurrentClass = (y==current_class).sum()      
            print("Count of ",current_class," = ",countOfCurrentClass)
            
            if countOfCurrentClass>= max_count :
                max_count = countOfCurrentClass
                output_class  = current_class
        
        
        print("Current Entropy is =",entropy(y))
        
        best_feature,gain_ratio = best_split_possible(x,y,features)
        print("Splitting on feature ",best_feature,"with gain ratio :",gain_ratio)
        print("Output Class Name =", output_class)
        print()
        
        values_in_best_feature = set(x[best_feature]) 
        
        for value in values_in_best_feature:
            
            xnew=x[(x[best_feature] == value)]
            ynew=y[(x[best_feature] == value)]

            # we'll update our feature set
            remaining_features = features - {best_feature}
            #recursion call with updates x,y,features and level
            DT(xnew,ynew,remaining_features,level+1)
        return
    #################################################
    
    

In [7]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a,b,c,d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')

df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)


In [8]:
y = pd.DataFrame(iris.target)
y[y[0] == 0] = 'setosa'
y[y[0] == 1] = 'versicolor'
y[y[0] == 2] = 'virginica'
features = set(df.columns)
y=y.values
DT(df, y, features, 0)

Level  0
Count of  setosa  =  50
Count of  versicolor  =  50
Count of  virginica  =  50
Current Entropy is = 1.584962500721156
Splitting on feature  pw_labeled with gain ratio : 0.699638203622209
Output Class Name = virginica

Level  1
Count of  versicolor  =  40
Count of  virginica  =  16
Current Entropy is = 0.863120568566631
Splitting on feature  pl_labeled with gain ratio : 0.4334099495621066
Output Class Name = versicolor

Level  2
Count of  versicolor  =  39
Count of  virginica  =  8
Current Entropy is = 0.6581912658132185
Splitting on feature  sl_labeled with gain ratio : 0.12674503775809332
Output Class Name = versicolor

Level  3
Count of  versicolor  =  23
Count of  virginica  =  7
Current Entropy is = 0.783776947484701
Splitting on feature  sw_labeled with gain ratio : 0.07092036405148876
Output Class Name = versicolor

Level  4
Count of  versicolor  =  6
Current Entropy is = 0.0
Reached leaf Node
Output Class Name = versicolor

Level   4
Count of  versicolor  =  14
Count of